In [1]:
import pandas as pd
import torch
from torch.utils.data import DataLoader

import numpy as np
import pandas as pd
import os
import cv2
import glob
import timm
from tqdm import tqdm
import random
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
from torchvision.transforms import ToTensor
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.metrics import average_precision_score

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [2]:
class CustomDataset(Dataset):
    def __init__(self, csv, mode):
        self.csv = csv
        self.mode = mode

    def __len__(self):
        return self.csv.shape[0]

    def __getitem__(self, idx):
        file = self.csv.loc[idx, 'file']
        label = torch.as_tensor(self.csv.loc[idx, 'label'], dtype=torch.long)
        file = np.load(file)

        return torch.Tensor(file).permute(2, 0, 1), label

In [3]:
class ResNet50GRUClassifier(nn.Module):
    def __init__(self, num_classes, hidden_size, num_layers):
        super(ResNet50GRUClassifier, self).__init__()

        self.resnet = timm.create_model('resnet50', pretrained=True)
        self.resnet = nn.Sequential(*list(self.resnet.children())[:-1])

        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.gru = nn.GRU(2048, hidden_size, num_layers, batch_first=True, dropout=0.5)

        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        features = self.resnet(x)
        features = features.view(features.size(0), -1)

        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        out, _ = self.gru(features.unsqueeze(1), h0)

        output = self.fc(out[:, -1, :])
        return output

In [4]:
num_classes = 8
hidden_size = 1280
num_layers = 2

In [95]:
model = ResNet50GRUClassifier(num_classes, hidden_size, num_layers).to(device)
model.load_state_dict(torch.load('./model/6round_8person_time.pt'))
model.to(device)
model.eval()

ResNet50GRUClassifier(
  (resnet): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (4): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (act1): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (drop_block): Identity()
        (act2): ReLU(inplace=True)
        (aa): Identity()
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentu

In [129]:
batch_size = 10

In [130]:
df = pd.read_csv('./8_longitudinal_spectrogram_time.csv')
df_valid = df[df['round'] == 6]
df_valid = df_valid.reset_index(drop=True)

In [131]:
valid_dataset = CustomDataset(df_valid, 'valid')
valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)

In [132]:
labels = []
probabilities = []

In [133]:
with torch.no_grad():
    for data, label in valid_loader:
        data = data.to(device)
        output = model(data)
        softmax_probs = F.softmax(output, dim=1).cpu().numpy()
        probabilities.extend(softmax_probs)
        labels.extend(label.numpy())

In [134]:
unique_labels = np.unique(labels)
average_probabilities = {label: np.mean([prob for l, prob in zip(labels, probabilities) if l == label], axis=0) for label in unique_labels}

In [135]:
avg_probs_df = pd.DataFrame(average_probabilities).transpose()
avg_probs_df

,0,1,2,3,4,5,6,7
0,0.172530,0.135233,0.100230,0.110680,0.143139,0.065333,0.173733,0.099122
1,0.132343,0.133703,0.131426,0.137849,0.140507,0.091757,0.130188,0.102227
2,0.114501,0.122734,0.123544,0.101167,0.112304,0.140063,0.120182,0.165504
3,0.123383,0.130423,0.136023,0.154569,0.145867,0.085609,0.125300,0.098826
4,0.075404,0.101254,0.111947,0.069378,0.080347,0.192661,0.101213,0.267797
5,0.099629,0.114244,0.124112,0.105280,0.109429,0.148655,0.112242,0.186408
6,0.166483,0.136426,0.108386,0.127922,0.152745,0.058196,0.165759,0.084083
7,0.117023,0.121658,0.126717,0.123774,0.125752,0.127126,0.122610,0.135340


In [136]:
avg_probs_df.columns = [f'{i}' for i in range(num_classes)]
avg_probs_df['val_6'] = avg_probs_df.index

In [137]:
avg_probs_df

,0,1,2,3,4,5,6,7,val_6
0,0.172530,0.135233,0.100230,0.110680,0.143139,0.065333,0.173733,0.099122,0
1,0.132343,0.133703,0.131426,0.137849,0.140507,0.091757,0.130188,0.102227,1
2,0.114501,0.122734,0.123544,0.101167,0.112304,0.140063,0.120182,0.165504,2
3,0.123383,0.130423,0.136023,0.154569,0.145867,0.085609,0.125300,0.098826,3
4,0.075404,0.101254,0.111947,0.069378,0.080347,0.192661,0.101213,0.267797,4
5,0.099629,0.114244,0.124112,0.105280,0.109429,0.148655,0.112242,0.186408,5
6,0.166483,0.136426,0.108386,0.127922,0.152745,0.058196,0.165759,0.084083,6
7,0.117023,0.121658,0.126717,0.123774,0.125752,0.127126,0.122610,0.135340,7


In [139]:
column_order = ['val_6'] + [col for col in avg_probs_df if col != 'val_6']
avg_probs_df = avg_probs_df[column_order]

In [140]:
# DataFrame 생성 및 CSV로 저장
avg_probs_df.to_csv('./prob/round_6_avg_prob.csv', index=False, float_format='%.3f')